# Lab-03 Q1 AlexNet

Please run the code with "VScode-devcontainer".

( It is easier to use Git in local. )

In [348]:
# install requirements

# !wget -P data/cifar10_1 https://github.com/modestyachts/CIFAR-10.1/raw/master/datasets/cifar10.1_v6_labels.npy
# !wget -P data/cifar10_1 https://github.com/modestyachts/CIFAR-10.1/raw/master/datasets/cifar10.1_v6_data.npy

In [349]:
# import required libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader
from pathlib import Path
import torch.nn as nn
import numpy as np
from pytorchcv.model_provider import get_model
from torchinfo import summary
from tqdm import tqdm

from utils import Cifar10_1

In [350]:
# define some hyper parameters
DATA_PATH = './data'

# change these hyper parameter if needed
BATCH_SIZE = 32
EPOCH = 10

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# check device (CPU, GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [351]:
# TODO
# augment data here
# https://pytorch.org/vision/master/transforms.html
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    # add augmentation here...
    transforms.RandomHorizontalFlip(),
])

In [352]:
# load dataset

# cifar10
trainset   = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset_1  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testset_2  = Cifar10_1(root='./data/cifar10_1/', transform=transform)
print(trainset.data.shape)
print(testset_1.data.shape)
print(testset_2.data.shape)

Files already downloaded and verified
Files already downloaded and verified
(50000, 32, 32, 3)
(10000, 32, 32, 3)
(2000, 32, 32, 3)


In [353]:
# TODO
# modify your training data here


In [354]:
# TODO
# split your training data if needed

In [355]:
# TODO
# select a model you want (small one or large one)
# https://github.com/osmr/imgclsmob/blob/master/pytorch/README.md
model_name = 'nin'
# model_name = 'densenet100_k24'
# model_name = 'preresnet56'
model = get_model(name = model_name + '_cifar10', pretrained=True)
model = model.to(device)

# choose loss function
LOSS_FUNCTION = "CrossEntropy"

In [356]:
summary(model, input_size=(1, 3, 32, 32), device=device)

Layer (type:depth-idx)                   Output Shape              Param #
CIFARNIN                                 [1, 10]                   --
├─Sequential: 1-1                        [1, 192, 8, 8]            --
│    └─Sequential: 2-1                   [1, 96, 32, 32]           --
│    │    └─NINConv: 3-1                 [1, 192, 32, 32]          14,592
│    │    └─NINConv: 3-2                 [1, 160, 32, 32]          30,880
│    │    └─NINConv: 3-3                 [1, 96, 32, 32]           15,456
│    └─Sequential: 2-2                   [1, 192, 16, 16]          --
│    │    └─MaxPool2d: 3-4               [1, 96, 16, 16]           --
│    │    └─Dropout: 3-5                 [1, 96, 16, 16]           --
│    │    └─NINConv: 3-6                 [1, 192, 16, 16]          460,992
│    │    └─NINConv: 3-7                 [1, 192, 16, 16]          37,056
│    │    └─NINConv: 3-8                 [1, 192, 16, 16]          37,056
│    └─Sequential: 2-3                   [1, 192, 8, 8]     

In [357]:
# TODO
# version control
UUID = 1

hyperparameter_log = {
  "UUID": UUID,
  "BATCH_SIZE": BATCH_SIZE,
  "EPOCH": EPOCH,
  "MODEL_NAME": model_name,
  "LOSS_FUNCTION": LOSS_FUNCTION,
  "DEVICE": device.type
}

print(hyperparameter_log)

{'UUID': 1, 'BATCH_SIZE': 32, 'EPOCH': 10, 'MODEL_NAME': 'nin', 'LOSS_FUNCTION': 'CrossEntropy', 'DEVICE': 'cuda'}


In [358]:
# dataloader
trainloader  = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader_1 = DataLoader(testset_1, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
testloader_2 = DataLoader(testset_2, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [359]:
# TODO
# define loss function
# https://blog.csdn.net/weixin_36670529/article/details/105670337
if LOSS_FUNCTION == "CrossEntropy":
  criterion = nn.CrossEntropyLoss()
elif LOSS_FUNCTION == "MSE":
  criterion = nn.MSELoss()

# or define your own loss function here
# https://discuss.pytorch.org/t/custom-loss-functions/29387
# https://rowantseng.medium.com/pytorch-%E8%87%AA%E5%AE%9A%E7%BE%A9%E6%90%8D%E5%A4%B1%E5%87%BD%E6%95%B8-custom-loss-c12e8741968b
# https://androidkt.com/how-to-add-l1-l2-regularization-in-pytorch-loss-function/
# 😂

# Do not modify this
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [360]:
# train
def train(e):
    model.train()
    num_data = 0
    correct = 0
    loss_all = 0

    for i, (x, y) in enumerate(tqdm(trainloader)):
        # get the inputs; data is a list of [inputs, labels]
        x, y = x.to(device), y.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(x)

        # compute loss here
        loss = criterion(outputs, y)

        # L2 Regularization
        # Reference: https://androidkt.com/how-to-add-l1-l2-regularization-in-pytorch-loss-function/
        l2_lambda = 0.00005
        l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
        loss += l2_lambda * l2_norm

        # back prop
        loss.backward()
        optimizer.step()

        # log
        num_data += y.size(0)
        loss_all += loss.item()
        pred = outputs.data.max(1)[1]
        correct += pred.eq(y.view(-1)).sum().item()

    print(f"epoch: [{e}], loss: {loss_all/len(trainloader):.4f}, acc: {correct/num_data:.4f}")


# start training
for e in range(EPOCH):
    train(e)

100%|██████████| 1563/1563 [00:13<00:00, 116.97it/s]


epoch: [0], loss: 0.3067, acc: 0.9390


100%|██████████| 1563/1563 [00:13<00:00, 118.20it/s]


epoch: [1], loss: 0.2819, acc: 0.9438


100%|██████████| 1563/1563 [00:13<00:00, 115.79it/s]


...

epoch: [7], loss: 0.2548, acc: 0.9527


100%|██████████| 1563/1563 [00:13<00:00, 118.20it/s]


epoch: [8], loss: 0.2528, acc: 0.9535


100%|██████████| 1563/1563 [00:13<00:00, 114.71it/s]

epoch: [9], loss: 0.2480, acc: 0.9553


In [361]:
# evaluate
def test(model, test_loader, loss_fun, device):
    model.eval()
    test_loss = 0
    correct = 0
    targets = []

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device).long()

        targets.append(target.detach().cpu().numpy())

        output = model(data)

        test_loss += loss_fun(output, target).item()
        pred = output.data.max(1)[1]

        correct += pred.eq(target.view(-1)).sum().item()

    return test_loss/len(test_loader), correct /len(test_loader.dataset)



loss, acc = test(model, testloader_1, criterion, device)
print(f"on testset 1: loss: {loss:.4f}, acc: {acc:.4f}")

loss, acc = test(model, testloader_2, criterion, device)
print(f"on testset 2: loss: {loss:.4f}, acc: {acc:.4f}")


on testset 1: loss: 0.2270, acc: 0.9254
on testset 2: loss: 0.4936, acc: 0.8465


In [362]:
# Save the model
torch.save(model.state_dict(), f'./model/model_{UUID}.pth')

In [363]:
# record the results to a log file (e.g google sheet, )